In [1]:
from graph.graphutils import flat_graph_formulation, VAR, COMP, SOLVER
from graph.graphutils import all_variables, edges_to_Ein_Eout, namefromid
from src.v1.symbolic import Var
from src.v2.execution import edges_from_components
from src.v2.transformations import generate_components_and_residuals
from graph.operators import sort_scc, reorder_merge_solve
from graph.workflow import get_f, OPT, NEQ, OBJ, order_from_tree, default_solver_options, mdao_workflow
from src.v2.workflow_mdao import mdao_workflow_with_args
from src.v2.mdaobuild import build_archi
from src.v2.runpipeline import nestedform_to_mdao
from src.v3.nesting import Model, adda, addf, setsolvefor
import openmdao.api as om
from sympy import exp

In [2]:
z1,z2,x,y2 = Var('z1'), Var('z2'), Var('x'), Var('y2')

In [3]:
model = Model(solver=OPT)
m = model.root
a = adda(m, 'a', z2+x-0.2*y2)
y1 = adda(m, 'y1', z1**2+a)
adda(m, y2, y1**0.5+z1+z2)
addf(m, x**2+z2+y1+exp(-y2)) #addobj
addf(m, 3.16-y1) #addineq
addf(m, y2-24) #addineq
setsolvefor(m, [x,z1,z2])

In [4]:
edges = edges_from_components(model.components)
tree = model.Ftree, model.Stree, model.Vtree

In [5]:
nodetyperepr = {VAR: '{}', COMP: 'f_{}', SOLVER: 's_{}'}

### OpenMDAO prob structure 1)

In [6]:
solvers_options = {1: {'type': OPT}}
comp_options = {3:OBJ, 4:NEQ, 5:NEQ}
var_options = {'x': [0,10], 'z1': [0,10], 'z2': [0,10]}

In [7]:
prob, mdao_in, groups, _,_ = nestedform_to_mdao(edges, tree, model.components, solvers_options, comp_options, var_options, nodetyperepr, mdf=True)

### OpenMDAO prob structure 2)

In [8]:
G = flat_graph_formulation(*edges)
merge_order = sort_scc(G)

In [9]:
nedges, ntree = reorder_merge_solve(edges, tree, merge_order, 1, True)
sequence = order_from_tree(ntree[0], ntree[1], nedges[1])
sequence

[(SOLVER, 1, None),
 (SOLVER, 2, 1),
 (ENDCOMP, [2, 0, 1], 2),
 (ENDCOMP, [3, 5, 4], 1)]

In [10]:
solvers_options = default_solver_options(ntree, solvers_options)

In [11]:
wf = mdao_workflow(sequence, solvers_options, comp_options, var_options)

In [12]:
components = generate_components_and_residuals(model.components, nedges)

In [13]:
lookup_f = get_f(components, nedges)

In [14]:
namingfunc = namefromid(nodetyperepr)

In [15]:
wfmdao = mdao_workflow_with_args(wf, lookup_f, namingfunc)

In [16]:
prob, mdao_in, groups = build_archi(nedges, ntree, wfmdao, namingfunc)

In [17]:
# prob.set_val('x', 1.0)
# prob.set_val('z1', 1.0)
# prob.set_val('z2', 1.0)

In [18]:
#prob.run_model()

In [19]:
prob.run_driver();


===
s_2
===
NL: Newton Converged in 3 iterations

===
s_2
===
NL: Newton Converged in 0 iterations

===
s_2
===
NL: Newton Converged in 3 iterations

===
s_2
===
NL: Newton Converged in 3 iterations

===
s_2
===
NL: Newton Converged in 3 iterations

===
s_2
===
NL: Newton Converged in 3 iterations

===
s_2
===
NL: Newton Converged in 2 iterations

===
s_2
===
NL: Newton Converged in 1 iterations
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.183393951639923
            Iterations: 5
            Function evaluations: 7
            Gradient evaluations: 5
Optimization Complete
-----------------------------------


In [20]:
allvars = all_variables(*edges_to_Ein_Eout(edges))

In [21]:
import numpy as np

In [33]:
xopt = {model.idmapping[key]: prob.get_val(namingfunc(key, VAR))[0] for key in allvars}

In [34]:
xopt

{z1: 1.9776388834629222,
 z2: 0.0,
 y1: 3.1599999999993003,
 y2: 3.7552777669258646,
 a: -0.7510555533851729,
 x: 0.0}

In [36]:
[comp.evaldict(xopt) for comp in components[0:6]]

[[DeviceArray(-0.75105555, dtype=float64)],
 [DeviceArray(3.16, dtype=float64)],
 [DeviceArray(3.75527777, dtype=float64)],
 DeviceArray(3.18339395, dtype=float64),
 6.998845947236987e-13,
 -20.244722233074135]